# Extraer planos nodales de sismos usando el "product type" de mecanismos focales en la página del NEIC
---

## Dependencias del código 

In [ ]:
from datetime import datetime
from libcomcat.search import search, get_event_by_id
from libcomcat.dataframes import get_summary_data_frame
import pandas as pd

## Parámetros de búsqueda (fecha, latitud, longitud y magnitud)

In [ ]:
starttime = datetime(2001, 9, 17, 0, 0, 0) #yyyy, mm, dd,hh, mm, ss
endtime = datetime(2001, 9, 17, 0, 0, 0)

min_latitude = 4.675
max_latitude = 12.147
min_longitude = -88.982
max_longitude = -81.101
min_mag = 5
max_mag = 8

events = search(starttime=starttime, endtime=endtime,
                maxlatitude=max_latitude, minlatitude=min_latitude,
                maxlongitude=max_longitude, minlongitude=min_longitude,
                minmagnitude=min_mag, maxmagnitude=max_mag,
                producttype='focal-mechanism')

event_data = []

for event in events:
    try:
        event_details = get_event_by_id(event.id)

        focal_mechanism_products = event_details.getProducts('focal-mechanism')
        
        if focal_mechanism_products:
            focal_mechanism_product = focal_mechanism_products[0]

            strike1 = focal_mechanism_product['nodal-plane-1-strike']
            dip1 = focal_mechanism_product['nodal-plane-1-dip']
            rake1 = focal_mechanism_product['nodal-plane-1-rake']

            strike2 = focal_mechanism_product['nodal-plane-2-strike']
            dip2 = focal_mechanism_product['nodal-plane-2-dip']
            rake2 = focal_mechanism_product['nodal-plane-2-rake']
            
            event_info = {
                'id': event.id,
                'time': event.time,
                'latitude': event.latitude,
                'longitude': event.longitude,
                'depth': event.depth,
                'magnitude': event.magnitude,
                'strike_1': strike1,
                'dip_1': dip1,
                'rake_1': rake1,
                'strike_2': strike2,
                'dip_2': dip2,
                'rake_2': rake2
            }

            event_data.append(event_info)
        else:
            print(f"Evento {event.id} no tiene un producto de tipo focal-mechanism")

    except Exception as e:
        print(f"Error al procesar evento {event.id}: {str(e)}")

## Ver lista de eventos procesados

In [ ]:
print("Lista de eventos:")
for event in event_data:
    print(event)

## Guardar la lista de eventos en un .csv

In [ ]:
events_df = pd.DataFrame(event_data)

# Acá se puede cambiar el nombre del archivo, este se guarda en la misma carpeta en la que se encuentra este notebook

csv_file = 'sismos_mecanismos_focales.csv' 
events_df.to_csv(csv_file, index=False)

print(f"Datos guardados en '{csv_file}'")